# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup #
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import pprint

import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

596

In [3]:
# cities

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
def get_weather(city, api_key):
    url = "http://api.openweathermap.org/data/2.5/weather"
    payload = {
        "appid": api_key,
        "q":city
    }
    r = requests.get(url,params=payload)
    if r.status_code == 200:
        return r.json()
    else: 
        return None
    
#     print(f"The city {city}'s weather is :{r.json()['weather']} ")


In [8]:
all_cities = []
for city in cities:
    result = get_weather(city,api_key)
    if result: 
        all_cities.append(result)
        

In [11]:
# pprint.pprint(all_cities,indent=4)
# for ct in all_cities: 
len(cities)  

596

In [27]:
print("Begining Data Retrieval")
print('-'*45)
for e in range(len(all_cities)):
#     result = all_cities[e]['weather'][0]['description']
    print(f"Procssing Record 1 of Set 1 | {all_cities[e]['name']}")

Begining Data Retrieval
---------------------------------------------
Procssing Record 1 of Set 1 | Qaanaaq
Procssing Record 1 of Set 1 | Busselton
Procssing Record 1 of Set 1 | Avarua
Procssing Record 1 of Set 1 | Albany
Procssing Record 1 of Set 1 | Vaini
Procssing Record 1 of Set 1 | Lebu
Procssing Record 1 of Set 1 | Jamestown
Procssing Record 1 of Set 1 | Port Alfred
Procssing Record 1 of Set 1 | Barrow
Procssing Record 1 of Set 1 | Tuktoyaktuk
Procssing Record 1 of Set 1 | Rikitea
Procssing Record 1 of Set 1 | Mar del Plata
Procssing Record 1 of Set 1 | Guerrero Negro
Procssing Record 1 of Set 1 | Dikson
Procssing Record 1 of Set 1 | Kailahun
Procssing Record 1 of Set 1 | Najran
Procssing Record 1 of Set 1 | Murray Bridge
Procssing Record 1 of Set 1 | Victoria
Procssing Record 1 of Set 1 | Phulji
Procssing Record 1 of Set 1 | Tawau
Procssing Record 1 of Set 1 | Puerto Ayora
Procssing Record 1 of Set 1 | Labuhan
Procssing Record 1 of Set 1 | Samarai
Procssing Record 1 of Set 1 | U

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot